# LLM Optimization Modelling Experiment

In [4]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [62]:
problem = '''Consider a farmer who specializes in raising wheat, corn, and sugar beets on her 500 acres of land. During the winter, she wants to decide how much land to devote to each crop.

The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler.
Any production in excess of the feeding requirement would be sold.

Over the last decade, mean selling prices have been $170 and $150 per ton of wheat and corn, respectively.
The purchase prices are 40% more than this due to the wholesaler’s margin and transportation costs.

Another profitable crop is sugar beet, which she expects to sell at $36/T. However, the European Commission imposes a quota on sugar beet production. Any amount in excess of the quota can be sold only at $10/T. The farmer’s quota for next year is 6000 T.

Based on past experience, the farmer knows that the mean yield on her land is roughly 2.5 T, 3 T, and 20 T per acre for wheat, corn, and sugar beets, respectively. It costs $150 to plant an acre of wheat, $230 to pant an acre of corn and $260 to plant an acre of sugar beets. 

The farmer can decide how much of her land she will use to grow each of the three products: wheat, corn and sugar beets. The goal of the farmer is to minimize her total costs (where sales are considered as negative costs).'''

## 2. Generate the mathematical model

In [75]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [76]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Farmer's Crop Planning

**Parameters:**

* $A$: Total land available (500 acres)
* $W_{demand}$: Wheat demand for cattle feed (200 T)
* $C_{demand}$: Corn demand for cattle feed (240 T)
* $S_{quota}$: Sugar beet quota (6000 T)
* $P_w$: Selling price of wheat ($170/T)
* $P_c$: Selling price of corn ($150/T)
* $P_{sb}$: Selling price of sugar beet within quota ($36/T)
* $P_{sb}^L$: Selling price of sugar beet exceeding quota ($10/T)
* $C_w$: Cost of planting wheat ($150/acre)
* $C_c$: Cost of planting corn ($230/acre)
* $C_{sb}$: Cost of planting sugar beets ($260/acre)
* $Y_w$: Yield of wheat (2.5 T/acre)
* $Y_c$: Yield of corn (3 T/acre)
* $Y_{sb}$: Yield of sugar beets (20 T/acre)

**Decision Variables:**

* $A_w$: Acres of land allocated to wheat 
* $A_c$: Acres of land allocated to corn
* $A_{sb}$: Acres of land allocated to sugar beets
* $W_{buy}$: Tons of wheat purchased
* $C_{buy}$: Tons of corn purchased
* $S_{high}$: Tons of sugar beets sold within quota 
* $S_{low}$: Tons of sugar beets sold exceeding quota

**Objective Function (Minimize Total Cost):**

```
Minimize Z =  C_w * A_w + C_c * A_c + C_{sb} * A_{sb} 
             + 1.4 * P_w * W_{buy} + 1.4 * P_c * C_{buy}
             - P_w * (Y_w * A_w + W_{buy} - W_{demand})
             - P_c * (Y_c * A_c + C_{buy} - C_{demand})
             - P_{sb} * S_{high} - P_{sb}^L * S_{low} 
```

**Constraints:**

* **Land availability:** 
   *  $A_w + A_c + A_{sb} \le A$
* **Wheat production and purchase:**
   *  $Y_w * A_w + W_{buy} \ge W_{demand}$ 
* **Corn production and purchase:**
   *  $Y_c * A_c + C_{buy} \ge C_{demand}$
* **Sugar beet production and sales:**
    * $S_{high} + S_{low} = Y_{sb} * A_{sb}$
    * $S_{high} \le S_{quota}$
* **Non-negativity:**
    * $A_w, A_c, A_{sb}, W_{buy}, C_{buy}, S_{high}, S_{low} \ge 0$ 

**Explanation:**

* **Objective function:** Represents the total cost, including planting costs, purchase costs (40% above selling price), and revenue from sales (negative cost).
* **Land availability constraint:** Ensures total allocated land doesn't exceed the available land.
* **Production and purchase constraints:** Ensure enough wheat and corn are available to meet the cattle feed demand, either through production or purchase.
* **Sugar beet constraints:** Define the relationship between sugar beet production and sales, ensuring the quota is respected.
* **Non-negativity constraints:** Ensure all decision variables are non-negative. 

This model allows the farmer to determine the optimal land allocation for each crop, the amount of wheat and corn to purchase, and the quantity of sugar beets to sell within and exceeding the quota to minimize her total costs. 


## 3. Generate the pyomo code

In [77]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [78]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Parameters
model.A = pyo.Param(initialize=500)  # Total land available (acres)
model.W_demand = pyo.Param(initialize=200)  # Wheat demand for cattle feed (T)
model.C_demand = pyo.Param(initialize=240)  # Corn demand for cattle feed (T)
model.S_quota = pyo.Param(initialize=6000)  # Sugar beet quota (T)

model.P_w = pyo.Param(initialize=170)  # Selling price of wheat ($/T)
model.P_c = pyo.Param(initialize=150)  # Selling price of corn ($/T)
model.P_sb = pyo.Param(initialize=36)  # Selling price of sugar beet within quota ($/T)
model.P_sb_L = pyo.Param(initialize=10)  # Selling price of sugar beet exceeding quota ($/T)

model.C_w = pyo.Param(initialize=150)  # Cost of planting wheat ($/acre)
model.C_c = pyo.Param(initialize=230)  # Cost of planting corn ($/acre)
model.C_sb = pyo.Param(initialize=260)  # Cost of planting sugar beets ($/acre)

model.Y_w = pyo.Param(initialize=2.5)  # Yield of wheat (T/acre)
model.Y_c = pyo.Param(initialize=3)  # Yield of corn (T/acre)
model.Y_sb = pyo.Param(initialize=20)  # Yield of sugar beets (T/acre)

# Decision Variables
model.A_w = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land allocated to wheat
model.A_c = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land allocated to corn
model.A_sb = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land allocated to sugar beets
model.W_buy = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of wheat purchased
model.C_buy = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of corn purchased
model.S_high = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of sugar beets sold within quota
model.S_low = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of sugar beets sold exceeding quota

# Objective Function
def objective_rule(model):
    return (model.C_w * model.A_w + model.C_c * model.A_c + model.C_sb * model.A_sb
            + 1.4 * model.P_w * model.W_buy + 1.4 * model.P_c * model.C_buy
            - model.P_w * (model.Y_w * model.A_w + model.W_buy - model.W_demand)
            - model.P_c * (model.Y_c * model.A_c + model.C_buy - model.C_demand)
            - model.P_sb * model.S_high - model.P_sb_L * model.S_low)

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.land_availability = pyo.Constraint(expr=model.A_w + model.A_c + model.A_sb <= model.A)

model.wheat_production = pyo.Constraint(expr=model.Y_w * model.A_w + model.W_buy >= model.W_demand)

model.corn_production = pyo.Constraint(expr=model.Y_c * model.A_c + model.C_buy >= model.C_demand)

model.sugar_beet_production = pyo.Constraint(expr=model.S_high + model.S_low == model.Y_sb * model.A_sb)

model.sugar_beet_quota = pyo.Constraint(expr=model.S_high <= model.S_quota)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose a different solver if you prefer
solver.solve(model)

# Print the results
print("Optimal Solution:")
print(f"A_w: {model.A_w.value:.2f} acres")
print(f"A_c: {model.A_c.value:.2f} acres")
print(f"A_sb: {model.A_sb.value:.2f} acres")
print(f"W_buy: {model.W_buy.value:.2f} tons")
print(f"C_buy: {model.C_buy.value:.2f} tons")
print(f"S_high: {model.S_high.value:.2f} tons")
print(f"S_low: {model.S_low.value:.2f} tons")
print(f"Total cost: ${model.objective.expr():.2f}")
```

**Sample Data:**

The sample data is already included in the code as the `initialize` values for the parameters. For example:

```python
model.A = pyo.Param(initialize=500)  # Total land available (acres)
```

This line sets the total land available (`model.A`) to 500 acres. The other parameters are initialized similarly in the code.


## 4. Input problem data and try running the generated code

In [79]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Parameters
model.A = pyo.Param(initialize=500)  # Total land available (acres)
model.W_demand = pyo.Param(initialize=200)  # Wheat demand for cattle feed (T)
model.C_demand = pyo.Param(initialize=240)  # Corn demand for cattle feed (T)
model.S_quota = pyo.Param(initialize=6000)  # Sugar beet quota (T)

model.P_w = pyo.Param(initialize=170)  # Selling price of wheat ($/T)
model.P_c = pyo.Param(initialize=150)  # Selling price of corn ($/T)
model.P_sb = pyo.Param(initialize=36)  # Selling price of sugar beet within quota ($/T)
model.P_sb_L = pyo.Param(initialize=10)  # Selling price of sugar beet exceeding quota ($/T)

model.C_w = pyo.Param(initialize=150)  # Cost of planting wheat ($/acre)
model.C_c = pyo.Param(initialize=230)  # Cost of planting corn ($/acre)
model.C_sb = pyo.Param(initialize=260)  # Cost of planting sugar beets ($/acre)

model.Y_w = pyo.Param(initialize=2.5)  # Yield of wheat (T/acre)
model.Y_c = pyo.Param(initialize=3)  # Yield of corn (T/acre)
model.Y_sb = pyo.Param(initialize=20)  # Yield of sugar beets (T/acre)

# Decision Variables
model.A_w = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land allocated to wheat
model.A_c = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land allocated to corn
model.A_sb = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land allocated to sugar beets
model.W_buy = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of wheat purchased
model.C_buy = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of corn purchased
model.S_high = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of sugar beets sold within quota
model.S_low = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of sugar beets sold exceeding quota

# Objective Function
def objective_rule(model):
    return (model.C_w * model.A_w + model.C_c * model.A_c + model.C_sb * model.A_sb
            + 1.4 * model.P_w * model.W_buy + 1.4 * model.P_c * model.C_buy
            - model.P_w * (model.Y_w * model.A_w + model.W_buy - model.W_demand)
            - model.P_c * (model.Y_c * model.A_c + model.C_buy - model.C_demand)
            - model.P_sb * model.S_high - model.P_sb_L * model.S_low)

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.land_availability = pyo.Constraint(expr=model.A_w + model.A_c + model.A_sb <= model.A)

model.wheat_production = pyo.Constraint(expr=model.Y_w * model.A_w + model.W_buy >= model.W_demand)

model.corn_production = pyo.Constraint(expr=model.Y_c * model.A_c + model.C_buy >= model.C_demand)

model.sugar_beet_production = pyo.Constraint(expr=model.S_high + model.S_low == model.Y_sb * model.A_sb)

model.sugar_beet_quota = pyo.Constraint(expr=model.S_high <= model.S_quota)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose a different solver if you prefer
solver.solve(model)

# Print the results
print("Optimal Solution:")
print(f"A_w: {model.A_w.value:.2f} acres")
print(f"A_c: {model.A_c.value:.2f} acres")
print(f"A_sb: {model.A_sb.value:.2f} acres")
print(f"W_buy: {model.W_buy.value:.2f} tons")
print(f"C_buy: {model.C_buy.value:.2f} tons")
print(f"S_high: {model.S_high.value:.2f} tons")
print(f"S_low: {model.S_low.value:.2f} tons")
print(f"Total cost: ${model.objective.expr():.2f}")

Optimal Solution:
A_w: 120.00 acres
A_c: 80.00 acres
A_sb: 300.00 acres
W_buy: 0.00 tons
C_buy: 0.00 tons
S_high: 6000.00 tons
S_low: 0.00 tons
Total cost: $-118600.00


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [80]:
print(response.text)

## Mathematical Optimization Model for Farmer's Crop Planning

**Parameters:**

* $A$: Total land available (500 acres)
* $W_{demand}$: Wheat demand for cattle feed (200 T)
* $C_{demand}$: Corn demand for cattle feed (240 T)
* $S_{quota}$: Sugar beet quota (6000 T)
* $P_w$: Selling price of wheat ($170/T)
* $P_c$: Selling price of corn ($150/T)
* $P_{sb}$: Selling price of sugar beet within quota ($36/T)
* $P_{sb}^L$: Selling price of sugar beet exceeding quota ($10/T)
* $C_w$: Cost of planting wheat ($150/acre)
* $C_c$: Cost of planting corn ($230/acre)
* $C_{sb}$: Cost of planting sugar beets ($260/acre)
* $Y_w$: Yield of wheat (2.5 T/acre)
* $Y_c$: Yield of corn (3 T/acre)
* $Y_{sb}$: Yield of sugar beets (20 T/acre)

**Decision Variables:**

* $A_w$: Acres of land allocated to wheat 
* $A_c$: Acres of land allocated to corn
* $A_{sb}$: Acres of land allocated to sugar beets
* $W_{buy}$: Tons of wheat purchased
* $C_{buy}$: Tons of corn purchased
* $S_{high}$: Tons of sugar beets so

In [81]:
print(response2.text)

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Parameters
model.A = pyo.Param(initialize=500)  # Total land available (acres)
model.W_demand = pyo.Param(initialize=200)  # Wheat demand for cattle feed (T)
model.C_demand = pyo.Param(initialize=240)  # Corn demand for cattle feed (T)
model.S_quota = pyo.Param(initialize=6000)  # Sugar beet quota (T)

model.P_w = pyo.Param(initialize=170)  # Selling price of wheat ($/T)
model.P_c = pyo.Param(initialize=150)  # Selling price of corn ($/T)
model.P_sb = pyo.Param(initialize=36)  # Selling price of sugar beet within quota ($/T)
model.P_sb_L = pyo.Param(initialize=10)  # Selling price of sugar beet exceeding quota ($/T)

model.C_w = pyo.Param(initialize=150)  # Cost of planting wheat ($/acre)
model.C_c = pyo.Param(initialize=230)  # Cost of planting corn ($/acre)
model.C_sb = pyo.Param(initialize=260)  # Cost of planting sugar beets ($/acre)

model.Y_w = pyo.Param(initialize=2.5)  # Yield of wheat (T/a